In [2]:
import jieba
s1 = "兴高采烈去退税 结果要补1万8"
s2 = "今天可以退税了，大家快去操作吧"
s3 = "新闻：你退税退了多少钱啊"

s4 = "工业机器人如何“显身手”？"
s5 = "国家政策，推进工业化，加快对机器人的研究"
c = jieba.cut(s)
list(c)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\QINGYU~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.360 seconds.
Prefix dict has been built successfully.


['今天', '有个', '大', '新闻']

## 中文文本聚类小例子

##### 文本向量化

In [ ]:
import pandas as pd
from nltk import FreqDist

df = pd.DataFrame()
sentence_list = [
    ['我', '的','狗','狗',"狗"],
    ['我','的','猫',"猫"],
    ['我','的','狗'],
    ['我','猫',"猫"]
    ]

for line in sentence_list:
    tmp_series = pd.Series(FreqDist(line))
    df = pd.concat([df, tmp_series.to_frame().T], ignore_index=True)
df

,我,的,狗,猫
0,1,1.0,3.0,NaN
1,1,1.0,NaN,2.0
2,1,1.0,1.0,NaN
3,1,NaN,NaN,2.0


In [ ]:
import pandas as pd
from nltk import FreqDist

df = pd.DataFrame()
sentence_list = [['我','狗','狗',"狗"],['我','猫',"猫"],['我','狗'],['我','猫',"猫"]]

for line in sentence_list:
    tmp_series = pd.Series(FreqDist(line))
    df = pd.concat([df, tmp_series.to_frame().T], ignore_index=True)
df.fillna(0, inplace=True)
df

,我,狗,猫
0,1,3.0,0.0
1,1,0.0,2.0
2,1,1.0,0.0
3,1,0.0,2.0


In [ ]:
%matplotlib auto
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(15,6))
ax0 = fig.add_subplot(111, projection='3d') 
ax0.scatter(df['我'], df['狗'], df['猫'])

Using matplotlib backend: TkAgg


### 词频逆文档频率(TF-IDF)
字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。
$$
 TF \cdot IDF
$$
TF词频(Term Frequency)，IDF反文档频率(Inverse Document Frequency)

#### 逆文档频率 
$$  
    idf=\log(\frac{文档总数}{该词在在多少个文档中出现过})
$$

##### 文档总数

In [ ]:
import numpy as np
# 文档总数 N
n = df.shape[0]
n

4

##### 词x在多少个文档中出现过 N(x)

In [ ]:
not_na = df.notna()
not_na

,我,的,狗,猫
0,True,True,True,False
1,True,True,False,True
2,True,True,True,False
3,True,False,False,True


In [ ]:
nx = not_na.sum()
nx

我    4
的    3
狗    2
猫    2
dtype: int64

##### 逆文档频率IDF


In [ ]:
log_n_nx = np.log(n/nx)
log_n_nx

我    0.000000
的    0.287682
狗    0.693147
猫    0.693147
dtype: float64

#### 词频逆文档频率 TF-IDF

In [ ]:
tf_idf = np.multiply(df, log_n_nx)
tf_idf

,我,的,狗,猫
0,0.0,0.287682,2.079442,NaN
1,0.0,0.287682,NaN,1.386294
2,0.0,0.287682,0.693147,NaN
3,0.0,NaN,NaN,1.386294


#### 缺失值填充

In [ ]:
tf_idf.fillna(0, inplace=True)
tf_idf

,我,的,狗,猫
0,0.0,0.287682,2.079442,0.000000
1,0.0,0.287682,0.000000,1.386294
2,0.0,0.287682,0.693147,0.000000
3,0.0,0.000000,0.000000,1.386294


#### 文本聚类

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=2)
model.fit_predict(tf_idf)
model.labels_

d:\Program Files\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


array([0, 1, 0, 1])

#### 聚类结果

In [ ]:
tf_idf.loc[:,"result"] = pd.Series(model.labels_)
tf_idf

,我,的,狗,猫,result
0,0.0,0.287682,2.079442,0.000000,0
1,0.0,0.287682,0.000000,1.386294,1
2,0.0,0.287682,0.693147,0.000000,0
3,0.0,0.000000,0.000000,1.386294,1


#### 文本分类

In [ ]:
train_x = tf_idf.iloc[:, 0:3]
train_x

,我,的,狗
0,0.0,0.287682,2.079442
1,0.0,0.287682,0.000000
2,0.0,0.287682,0.693147
3,0.0,0.000000,0.000000


In [ ]:
train_y = tf_idf.iloc[:, 4]
train_y

0    0
1    1
2    0
3    1
Name: result, dtype: int32

In [ ]:
# 给定一段文本，使用KNN对它分类
# test_data = ['我','有','一','只','猫']

from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=2).fit(train_x, train_y)
# knn_model.predict(test_data)

改善

为什么TF要进行标准化操作？
解决长文档、短文档问题.


为什么要取对数？
使用log可以防止权重爆炸。
利用对数来达到非线性增长的目的.

为什么IDF分母中要进行+1（IDF如何进行平滑处理的）？
为了避免分母为 0（即所有文档都不包含该词）